In [25]:
import cv2
import os
import numpy as np

In [93]:
def display_img(image):
    
    cv2.imshow("Image", image)
    
    # Wait for the user to press a key
    cv2.waitKey(0)
    
    # Close all windows
    cv2.destroyAllWindows()



def rem_black_frame(image, threshold = 20):

    # 1. Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 2. Binarize the images with a low threshold
    _, binary_image = cv2.threshold(gray_image, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # 3. finding contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)

    # 4. obtain the bounding rectangle from the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)

    # 5. Determine the necessecity for cropping based on whether the mean intensity inside the bounding box is 
    # different from the mean intensity outside of the bounding box.
    mean_intensity_inside = np.mean(gray_image[y:y+h, x:x+w])
    
    # the denomentator should not be equal to 0, if so, it means the bounding rectangle has the same dimensions of 
    # the original image, thus not necessary to crop
    denominator = gray_image.size - w * h
    if denominator != 0:
        # Calculate the mean intensity outside the bounding box
        mean_intensity_outside = (np.mean(gray_image) * gray_image.size - np.mean(gray_image[y:y+h, x:x+w]) * w * h) / denominator

        # Check if cropping is necessary based on the mean intensity comparison
        if mean_intensity_inside - mean_intensity_outside > threshold:
            print("Cropping is necessary")

            # for debugging, display the image with the contour 
            # contour_image = cv2.drawContours(image.copy(), [largest_contour], -1, (255, 255, 255), 3)

            # crop the image using the bounding rectangle
            cropped_image = image.copy()[y:y+h, x:x+w]

            # TODO: Region filling

        else:
            print("Cropping is not necessary, use the original image")
            cropped_image = image
    else:
        print("Cropping is not necessary, use the original image")
        cropped_image = image

    
    display_img(cropped_image)

    return image


# Samples to test (need cropping)
# nev00002
# nev00019
# nev00031

# Samples to test (doesn't need cropping, should be returned the same)
# nev00001
# nev00148
# nev00292
# nev00424
# nev01052
# nev01051

img_name = 'nev01476.jpg'
img_dir  = os.path.join(os.getcwd(), '../challenge1/train/nevus', img_name)

image = cv2.imread(img_dir,cv2.IMREAD_COLOR)

image = rem_black_frame(image)





Cropping is necessary


In [ ]:
# Test and try to obtain results as in the papers